In [0]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy

In [0]:
nlp=spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS
print(STOP_WORDS)

{'made', 'their', 'through', 'all', 'same', 'thru', "'ll", 'could', 'beforehand', 'as', 'hundred', 'last', 'by', 'with', 'too', 'when', 'i', 'became', 'few', 'had', 'may', 'down', 'therein', 'which', 'seem', 'front', 'several', 'well', 'more', 'towards', 'whereas', 'toward', 'many', 'therefore', 'third', 'yours', '‘re', 'whither', 'against', 'anyone', '‘ll', 'is', 'under', 'from', 'thereby', '‘s', 'any', 'do', 'thereupon', 'often', 'on', 'call', 'becoming', 'go', 'himself', 'via', 'get', 'into', 'whereupon', "'re", 'most', 'n‘t', 'before', 'n’t', 'without', 'anyhow', 'we', 'whole', 'almost', 'they', 'top', 'hence', 'for', 'another', 'everyone', 'whom', 'although', 'three', 'thence', 're', 'out', 'whereafter', 'been', 'only', 'keep', 'seems', 'either', 'over', 'indeed', 'whether', 'up', 'the', 'neither', 'should', 'onto', 'would', 'eleven', 'to', 'afterwards', 'two', 'an', 'perhaps', 'myself', 'both', 'can', 'your', 'quite', 'are', 'nine', 'yourselves', "'ve", 'every', 'did', 'must', 'b

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c nlp-getting-started

train.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [0]:
df=pd.read_csv("/content/train.csv",sep=',')
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [0]:
df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [0]:
import string
punct=string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
def text_cleaning(sentence):
  doc=nlp(sentence)
  tokens=[]
  for token in doc:
    if token.lemma_!="-PRON-":
      
      temp=token.lemma_.lower().strip()
      tokens.append(temp)
      #print(temp)
    else:
      temp=token.lower_
      tokens.append(temp)
  cleaned_tokens=[]
  #print(tokens)
  for token in tokens:
    if token not in STOP_WORDS and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens       

In [0]:
text_cleaning("hello,how are you? like this photo")

['hello', 'like', 'photo']

In [0]:
from sklearn.svm import LinearSVC
tfidf=TfidfVectorizer(tokenizer=text_cleaning)
classifier=LinearSVC()

In [0]:
X=df["text"]
y=df["target"]


In [0]:
clf=Pipeline([('tfidf',tfidf),('clf',classifier)])

In [0]:
clf.fit(X,y)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_cleaning at 0x7f7291740268>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True

In [0]:
df1=pd.read_csv("/content/test.csv",sep=',')
df1

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [0]:
X_test=df1["text"]
y_test=clf.predict(X_test)

In [0]:
y_test

array([1, 1, 1, ..., 1, 1, 1])

In [0]:
df1["target"]=y_test
df1=df1.drop(["keyword","location","text"],axis=1)
df1

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [0]:
df1.to_csv('submission1.csv', index = False)